In [1]:
import time
import numpy as np
import pandas as pd
import pickle
from scipy import sparse
from sklearn.feature_extraction.text import TfidfTransformer
import torch
from torchmetrics import MetricCollection, Precision, Recall, F1, Accuracy, AUROC
from torch.nn.functional import relu, sigmoid, cross_entropy, binary_cross_entropy, softmax
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader
my_random_state = 13


In [2]:
from ann import NeuralNet

In [4]:
model = torch.load('ann_model.pt')

In [ ]:
tfidf_transformer_file = open('../Data/Processed/tfidf_transformer.pkl', 'rb')
tfidf_transformer = pickle.load(tfidf_transformer_file)
print(tfidf_transformer.idf_.shape)
input_shape = tfidf_transformer.idf_.shape

In [ ]:
model = NeuralNet()

In [ ]:
trainer = pl.Trainer(max_epochs=1, gpus=1, fast_dev_run=False)

In [ ]:
# trainer.fit(model)

In [ ]:
# torch.save(model, 'ann_model.pt')

In [ ]:
model = torch.load('ann_model.pt')

In [ ]:
count_vec_file = open(f'../Data/Processed/count_vectorizer.pkl', mode='rb')
count_vec = pickle.load(count_vec_file)
count_vec_idx_map = {v:k for k,v in count_vec.vocabulary_.items()}
tfidf_trans_file = open(f'../Data/Processed/tfidf_transformer.pkl', mode='rb') 
tf_idf_trans = pickle.load(tfidf_trans_file)

In [ ]:
def transform_input_tfidf(sentence):
    if isinstance(sentence, str):
        sentence = [sentence]
    counts_vec = count_vec.transform(sentence)
    tfidf_vec = tf_idf_trans.transform(counts_vec)
    tfidf_tensor = make_torch_sparse_tensor(tfidf_vec.tocoo()).to_dense()
    return tfidf_tensor
            

In [ ]:
def transform_attribution(input_raw, attribution):
    counts_vec = count_vec.transform([input_raw])
    _, word_idxs = np.nonzero(counts_vec)
    print(f'word_idxs {word_idxs}')
    attr = attribution.squeeze(dim=0)[word_idxs]
    print(f'transformed attr: {attr}')
    print(f'transformed attr size: {attr.size()}')
    return attr

In [ ]:
def process_for_record(input_raw, model_out, target, attribution):
#     input_raw = input_raw.split() if not isinstance(input_raw, list) else input_raw
    pred_class = 'positive' if model_out > .5 else 'negative'
    true_class = 'positive' if model_out > .5 else 'negative'
    attr_score = attribution.sum() #attribution expected to be a tensor
    attr_class = 'positive' if attr_score > .5 else 'negative'
    convergence_score = torch.zeros(1, dtype=torch.float64)
    print(f'input_raw: {input_raw}')
    print(f'pred_class: {pred_class}')
    print(f'true_class: {true_class}')
    print(f'attr_class: {attr_class}')
    print(f'attr_score: {attr_score}')
    print(f'convergence_score: {convergence_score}')
    return pred_class, true_class, attr_class, attr_score, convergence_score
    

In [ ]:
# i = np.random.randint(low=0, high=Test_df.shape[0], size=10)
i = [15674] #, 15592, 10816,  1437,  6777, 22051, 18735,  3040,  7320, 15611]
rev = Train_df.loc[i, 'review']
rev_tfidf = transform_input_tfidf(rev)
# rev_targets = torch.tensor(list(Train_df.loc[i, 'sentiment']), dtype=torch.int64).unsqueeze(dim=1)
rev_targets = list(Train_df.loc[i, 'sentiment'])
print(f'rev: {rev}')
print(f'-------------------------------------')
print(f'rev_tfidf: {rev_tfidf.size()}')
print(f'-------------------------------------')
# print(f'rev_targets: {rev_targets.size()}')
print(f'-------------------------------------')

In [ ]:
from captum.attr import visualization 
# visualization.VisualizationDataRecord, visualize_text

In [ ]:
from captum.attr import Lime
lime = Lime(model)

In [ ]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_input(model, framework, input_raw, input_process_func, record_process_func, target):
    print(input_raw)
    input_processed = input_process_func(input_raw)
    print(input_processed)
    attribution_all = framework.attribute(inputs=input_processed, target=target)
    attribution_all_squeezed = attribution_all.squeeze(0)
    attr = attribution_all_squeezed[attribution_all_squeezed.nonzero()]
    print(f'attr: {attr}')
    print(f'attr size {attr.size()}')
    print(F'attribution all non zero {torch.count_nonzero(attribution_all, dim=1)}')
    to_feed = X_train_tfidf[i]
    model_out = model(to_feed)
    pred_prob = model_out.item()
    print(f'pred_prob type {type(pred_prob)}')
    pred_class, true_class, attr_class, attr_score, convergence_score = \
                            process_for_record(input_raw, model_out, target, attr)
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=torch.tensor([.1, .2, .3, .4], dtype=torch.float64),
                                                          pred_prob=pred_prob,
                                                          pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=['it', 'was', 'actually', 'bad'],
                                                         convergence_score=convergence_score)
    return visual_record
#     # input_indices dim: [sequence_length]
#     seq_length = min_len

#     # predict
#     pred = forward_with_sigmoid(input_indices).item()
#     pred_ind = round(pred)

#     # generate reference indices for each sample
#     reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

#     # compute attributions and approximation delta using layer integrated gradients
#     attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
#                                            n_steps=500, return_convergence_delta=True)

#     print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

#     add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, pred_prob, pred_class, true_class, pred, pred_ind, label, delta, vis_data_records):
#     attributions = attributions.sum(dim=2).squeeze(0)
#     attributions = attributions / torch.norm(attributions)
#     attributions = attributions.cpu().detach().numpy()
    print(f'attr: {attributions}')
#     print(f'--------------------------------')
#     print(f'pred_prob: {pred}')
#     print(f'-------------------------------')
#     print(f'pred_class {Label.vocab.itos[pred_ind]}')
#     print(f'--------------------------------')
#     print(f'true_class {Label.vocab.itos[label]}')
#     print(f'--------------------------------')
#     print(f'attr_class {Label.vocab.itos[1]}')
#     print(f'--------------------------------')
#     print(f'attr_score {attributions.shape}')
#     print(f'{attributions.sum()}')
#     print(f'--------------------------------')
#     print(f'raw_input {text}')
#     print(f'--------------------------------')
#     print(f'convergence_score {delta}')
#     delta = torch.zeros(1, dtype=torch.float64)
#     print(f'--------------------------------')
#     # storing couple samples in an array for visualization purposes
#     vis_data_records.append(visualization.VisualizationDataRecord(
#                             attributions,
#                             pred,
#                             Label.vocab.itos[pred_ind],
#                             Label.vocab.itos[label],
#                             Label.vocab.itos[1],
#                             attributions.sum(),
#                             text,
#                             delta))

In [ ]:
record = interpret_input(model=model, 
                framework=lime, 
                input_raw=['this review was really good'],
                input_process_func=transform_input_tfidf, 
                record_process_func=process_for_record,
                target=0)

In [ ]:
print(visualization.visualize_text([record]))

In [ ]:
rev_targets